In [65]:
%matplotlib inline
from PIL import Image
import pytesseract
from tqdm.notebook import tqdm
from pathlib import Path
import warnings
import pickle

In [66]:
PATH_TESSERACT,LANG = ("/deezy_datadrive/kaspar-playground/tesseract/tesstrain/data","npd_2")
tessdata_dir_config = f'--tessdata-dir "{PATH_TESSERACT}"  --psm 4'

In [67]:
with open('../editions_all.pickle','rb') as in_pickle:
    editions = pickle.load(in_pickle)

In [68]:
editions

{1846: [(39, 106), (111, 212), (213, 300)],
 1847: [(69, 140), (151, 263), (265, 357)],
 1851: [(79, 147), (157, 275), (277, 381)],
 1855: [(19, 35), (35, 65), (65, 86)],
 1856: [(21, 35), (35, 71), (71, 96)],
 1857: [(19, 33), (34, 72), (73, 94)],
 1858: [(19, 34), (35, 75), (75, 101)],
 1860: [(21, 36), (37, 84), (85, 112)],
 1861: [(29, 46), (47, 98), (99, 127)],
 1862: [(22, 40), (41, 96), (97, 126)],
 1863: [(23, 40), (41, 95), (96, 124)],
 1864: [(15, 28), (29, 85), (86, 115)],
 1865: [(15, 27), (29, 87), (88, 116)],
 1866: [(19, 31), (33, 92), (93, 120)],
 1867: [(26, 39), (40, 108), (109, 135)],
 1868: [(17, 31), (32, 95), (96, 122)],
 1869: [(19, 33), (34, 99), (100, 126)],
 1870: [(17, 31), (32, 100), (101, 126)],
 1871: [(16, 31), (32, 101), (102, 125)],
 1872: [(24, 37), (38, 108), (109, 131)],
 1873: [(21, 35), (36, 107), (108, 131)],
 1874: [(25, 38), (39, 112), (113, 136)],
 1875: [(22, 37), (38, 113), (114, 137)],
 1877: [(22, 38), (39, 124), (125, 150)],
 1878: [(26, 4

In [69]:
def generate_paths(editions,
                   column=True,
                   source='Image_processed_final',
                   root=Path('/deezy_datadrive/kaspar-playground/npd/Data/Original')):
    
    
    
    def generate_path(year,page,column=column): 
        if column:
            return [
                root / f'MPD_{year}/{source}/MPD_{year}_{str(page).zfill(3)}_right.png',
                root / f'MPD_{year}/{source}/MPD_{year}_{str(page).zfill(3)}_left.png'
            ]
        
        return [root / f'MPD_{year}/{source}/MPD_{year}_{str(page).zfill(3)}.png']
    
    paths = []
    for year,pranges in editions.items():
        for prange in pranges:
            paths.extend([f for p in range(*prange) for f in generate_path(year,p)])
            
    return [p for p in paths if p.is_file()]


In [57]:
editions_old =  {k:v for k,v in editions.items() if k < 1855}
image_paths_old = generate_paths(editions_old, column=False)

In [58]:
len(image_paths_old)

819

In [60]:
editions_new =  {k:v for k,v in editions.items() if k >= 1855}
image_paths_new = generate_paths(editions_new, column=True)

In [61]:
len(image_paths_new)

14664

In [ ]:
image_path = image_paths_old + image_paths_new

In [43]:
len(image_paths)

14206

In [73]:
warnings.filterwarnings('ignore')

for im_p in tqdm(image_paths): 
    (im_p.parent.parent / 'Plain Text').mkdir(exist_ok=True)
    
    txt_path = im_p.parent.parent / 'Plain Text' / (im_p.stem + ".txt")
    if txt_path.is_file(): continue
    image = Image.open(im_p)
    tessout = pytesseract.image_to_string(image, lang=LANG, 
                                          config=tessdata_dir_config)
    with open(txt_path,'wt') as txt_out:
             txt_out.write(tessout)


## Fin.